<a href="https://colab.research.google.com/github/AnyByte/iu5_bi_course/blob/master/lab2/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа №2 по дисциплине «Инструменты бизнес-аналитики» 

## 1 задача.
Есть N бандитов, для каждого есть число `-100<x<100`, которое подаётся на вход функции `pullBandit`
```
def pullBandit(bandit): #Сгенерировать случайное число
   result = np.random.randn(1) 
   if result > bandit: #Выигрыш 
        return 1 
   else: #Проигрыш 
        return -1
```
Значение результата используется в качестве награды.
Политика RL: `policy gradient` (лучше если это будет vanilla, без модификаций)

Задача найти номер лучшего бандита

In [0]:
import tensorflow.compat.v1 as tf
import numpy as np

In [0]:
tf.disable_v2_behavior() 

### Bandits

In [0]:
#Bandit list
bandits = [0.2,0,-0.2,-5]
num_bandits = len(bandits)
def pullBandit(bandit):
    #Get random number.
    result = np.random.randn(1)
    if result > bandit:
        #return a positive reward.
        return 1
    else:
        #return a negative reward.
        return -1

### Agent

In [0]:
tf.reset_default_graph()

weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights,0)

reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

### Agent training

In [15]:
total_episodes = 1000 #Episodes.
total_reward = np.zeros(num_bandits) #Fill scoreboard with 0.
e = 0.1 #Random action chance.

init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
        
        reward = pullBandit(bandits[action])
        
        _,resp,ww = sess.run([update,responsible_weight,weights], feed_dict={reward_holder:[reward],action_holder:[action]})
        
        total_reward[action] += reward
        if i % 50 == 0:
            print(f"Считаем награды для {num_bandits} бандитов: {total_reward}")
        i+=1
print(f"Агент считает, что бандит под номером {np.argmax(ww)+1} самый выгодный.")
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print("Агент оказался прав.")
else:
    print("Агент оказался неправ.")

Считаем награды для 4 бандитов: [1. 0. 0. 0.]
Считаем награды для 4 бандитов: [ 0. -1. -3. 41.]
Считаем награды для 4 бандитов: [ 0. -1. -2. 86.]
Считаем награды для 4 бандитов: [ -1.  -2.  -1. 133.]
Считаем награды для 4 бандитов: [ -2.  -3.  -1. 181.]
Считаем награды для 4 бандитов: [ -4.  -3.   0. 228.]
Считаем награды для 4 бандитов: [ -4.   0.   0. 269.]
Считаем награды для 4 бандитов: [ -3.  -2.   1. 315.]
Считаем награды для 4 бандитов: [ -4.  -3.   1. 361.]
Считаем награды для 4 бандитов: [ -4.  -4.   3. 406.]
Считаем награды для 4 бандитов: [ -5.  -3.   3. 454.]
Считаем награды для 4 бандитов: [ -6.  -3.   1. 499.]
Считаем награды для 4 бандитов: [ -8.  -2.   2. 545.]
Считаем награды для 4 бандитов: [-10.  -2.   2. 593.]
Считаем награды для 4 бандитов: [-10.  -1.   3. 639.]
Считаем награды для 4 бандитов: [-11.  -2.   2. 684.]
Считаем награды для 4 бандитов: [-12.  -2.   2. 731.]
Считаем награды для 4 бандитов: [-13.   0.   2. 778.]
Считаем награды для 4 бандитов: [-14.   0.  